In [1]:
from py2neo import Graph 
import pandas as pd
import re
import os
os.getcwd()

'D:\\pgkb_graph'

In [2]:
session = Graph("neo4j://172.16.227.27:7687", auth=("neo4j", "123456"))

In [29]:
haplotype_guideline_cypher = """
MATCH (hap:haplotype)-[r:variant_guideline]->(che:chemical) 
WITH hap, r, che
MATCH (hap)-[]->(ge:gene)
RETURN ge.gene_name as gene_name, hap.variant_name as haplotype_name, hap.is_reference as is_reference,
hap.NC_change_code as NC_code, hap.NG_change_code as NG_code, hap.nucleotide_change_code as nucleotide_code, 
hap.protein_change_code as protein_code, hap.mapped_rsID as mapped_rsID, 
r.phenotype as phenotype, r.implication as implication, r.implication_chn as implication_chn, 
r.recommendation as recommendation, r.recommendation_chn as recommendation_chn,
r.genotype as genotype, r.genotype_chn as genotype_chn,
r.organization as organization, r.link as link, 
che.chemical_name as chemical_name, che.atc_code as atc_code, 
che.L1_info as L1_info, che.L1_info_chn as L1_info_chn,
che.L2_info as L2_info, che.L2_info_chn as L2_info_chn,
che.L3_info as L3_info, che.L3_info_chn as L3_info_chn,
che.L4_info as L4_info, che.L4_info_chn as L4_info_chn
"""

result_list = session.run(haplotype_guideline_cypher).data()
df_haplotype_guideline = pd.DataFrame(result_list)

In [30]:
df_haplotype_guideline[:2]

,gene_name,haplotype_name,is_reference,NC_code,NG_code,nucleotide_code,protein_code,mapped_rsID,phenotype,implication,...,chemical_name,atc_code,L1_info,L1_info_chn,L2_info,L2_info_chn,L3_info,L3_info_chn,L4_info,L4_info_chn
0,RYR1,RYR1 c.103T>C,False,Position at NC_000019.10 (Homo sapiens chromos...,Position at NG_008866.1 (RYR1 RefSeqGene; forw...,NM_000540.2; ['c.103T>C'],Effect on protein (NP_000531.2); ['C35R'],rs193922747,Malignant Hyperthermia susceptible,Individuals are at increased risk of developin...,...,desflurane,N01AB07,N: NERVOUS SYSTEM,N：神经系统,N01: ANESTHETICS,N01：麻醉品,"N01A: ANESTHETICS, GENERAL",N01A：麻醉品，一般,N01AB: Halogenated hydrocarbons,N01AB：卤化碳氢化合物
1,RYR1,RYR1 c.130C>T,False,Position at NC_000019.10 (Homo sapiens chromos...,Position at NG_008866.1 (RYR1 RefSeqGene; forw...,NM_000540.2; ['c.130C>T'],Effect on protein (NP_000531.2); ['R44C'],rs193922748,Malignant Hyperthermia susceptible,Individuals are at increased risk of developin...,...,desflurane,N01AB07,N: NERVOUS SYSTEM,N：神经系统,N01: ANESTHETICS,N01：麻醉品,"N01A: ANESTHETICS, GENERAL",N01A：麻醉品，一般,N01AB: Halogenated hydrocarbons,N01AB：卤化碳氢化合物


In [31]:
diplotype_guideline_cypher = """
MATCH (dip:diplotype)-[r:variant_guideline]->(che:chemical) 
WITH dip, r, che
MATCH (dip)-[:diplotype_consist_of]->(hap:haplotype)-[:mutation_at]->(ge:gene)
RETURN distinct ge.gene_name as gene_name,
dip.diplotype_name as diplotype_name, r.phenotype as phenotype, 
hap.variant_name as haplotype_name, hap.is_reference as is_reference, hap.NC_change_code as NC_code, hap.NG_change_code as NG_code, 
hap.nucleotide_change_code as nucleotide_code, hap.protein_change_code as protein_code, hap.mapped_rsID as mapped_rsID,
r.implication as implication, r.implication_chn as implication_chn, 
r.recommendation as recommendation, r.recommendation_chn as recommendation_chn, 
r.genotype as genotype, r.genotype_chn as genotype_chn,
r.organization as organization, r.link as link, 
che.chemical_name as chemical_name, che.atc_code as atc_code, 
che.L1_info as L1_info, che.L1_info_chn as L1_info_chn,
che.L2_info as L2_info, che.L2_info_chn as L2_info_chn,
che.L3_info as L3_info, che.L3_info_chn as L3_info_chn,
che.L4_info as L4_info, che.L4_info_chn as L4_info_chn
"""

result_list = session.run(diplotype_guideline_cypher).data()
df_diplotype_guideline = pd.DataFrame(result_list)

In [32]:
df_diplotype_guideline[:2]

,gene_name,diplotype_name,phenotype,haplotype_name,is_reference,NC_code,NG_code,nucleotide_code,protein_code,mapped_rsID,...,chemical_name,atc_code,L1_info,L1_info_chn,L2_info,L2_info_chn,L3_info,L3_info_chn,L4_info,L4_info_chn
0,HLA-B,HLA-B *57:01/*X,High risk of hypersensitivity (~6% of patients),HLA-B*57:01,None,,,,,,...,abacavir,J05AF06,J: ANTIINFECTIVES FOR SYSTEMIC USE,J：系统使用的抗助液,J05: ANTIVIRALS FOR SYSTEMIC USE,J05：用于系统性使用的抗病毒,J05A: DIRECT ACTING ANTIVIRALS,J05A：直接代理抗病毒,J05AF: Nucleoside and nucleotide reverse trans...,J05AF：核苷和核苷酸逆转录酶抑制剂
1,HLA-B,HLA-B *57:01/*57:01,High risk of hypersensitivity (~6% of patients),HLA-B*57:01,None,,,,,,...,abacavir,J05AF06,J: ANTIINFECTIVES FOR SYSTEMIC USE,J：系统使用的抗助液,J05: ANTIVIRALS FOR SYSTEMIC USE,J05：用于系统性使用的抗病毒,J05A: DIRECT ACTING ANTIVIRALS,J05A：直接代理抗病毒,J05AF: Nucleoside and nucleotide reverse trans...,J05AF：核苷和核苷酸逆转录酶抑制剂


In [33]:
diplotype_metabolizer_cypher = """
MATCH (dip:diplotype)-[r:diplotype_metabolizer]->(che:chemical) 
WITH dip, r, che
MATCH (dip)-[:diplotype_consist_of]->(hap:haplotype)-[:mutation_at]->(ge:gene)
RETURN distinct ge.gene_name as gene_name,
dip.diplotype_name as diplotype_name, r.phenotype as phenotype, r.phenotype_category as metabolizer,
hap.variant_name as haplotype_name, hap.is_reference as is_reference, hap.NC_change_code as NC_code, 
hap.NG_change_code as NG_code, hap.nucleotide_change_code as nucleotide_code, 
hap.protein_change_code as protein_code, hap.mapped_rsID as mapped_rsID,
r.implication as implication, r.implication_chn as implication_chn,
r.recommendation as recommendation, r.recommendation_chn as recommendation_chn,
r.genotype as genotype, 
r.organization as organization, r.link as link, 
che.chemical_name as chemical_name, che.atc_code as atc_code, 
che.L1_info as L1_info, che.L1_info_chn as L1_info_chn,
che.L2_info as L2_info, che.L2_info_chn as L2_info_chn,
che.L3_info as L3_info, che.L3_info_chn as L3_info_chn,
che.L4_info as L4_info, che.L4_info_chn as L4_info_chn
"""
result_list = session.run(diplotype_metabolizer_cypher).data()
df_diplotype_metabolizer = pd.DataFrame(result_list)

In [34]:
df_diplotype_metabolizer[:2]

,gene_name,diplotype_name,phenotype,metabolizer,haplotype_name,is_reference,NC_code,NG_code,nucleotide_code,protein_code,...,chemical_name,atc_code,L1_info,L1_info_chn,L2_info,L2_info_chn,L3_info,L3_info_chn,L4_info,L4_info_chn
0,TPMT,TPMT *1/*2,"IM (one inactive allele: *2, *3, *4-*18)",Intermediate Metabolizer,TPMT*2,None,Position at NC_000006.12 (Homo sapiens chromos...,Position at NG_012137.2 (TPMT RefSeqGene); ['g...,Nucleotide change on cDNA from http://www.imh....,Effect on protein (NP_000358.1); ['A80P'],...,mercaptopurine,L01BB02,L: ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS,L：抗肿瘤和免疫调节剂,L01: ANTINEOPLASTIC AGENTS,L01：抗肿瘤剂,L01B: ANTIMETABOLITES,L01B：抗体代谢物,L01BB: Purine analogues,L01BB：嘌呤类似物
1,TPMT,TPMT *1/*2,"IM (one inactive allele: *2, *3, *4-*18)",Intermediate Metabolizer,TPMT*1,None,,,,,...,mercaptopurine,L01BB02,L: ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS,L：抗肿瘤和免疫调节剂,L01: ANTINEOPLASTIC AGENTS,L01：抗肿瘤剂,L01B: ANTIMETABOLITES,L01B：抗体代谢物,L01BB: Purine analogues,L01BB：嘌呤类似物


In [36]:
def get_position(nc_code):
    find_list = re.findall(r"chromosome [\d]+", nc_code)
    if len(find_list) > 0:
        chr_name = find_list[0].replace("chromosome ", "chr")
        position_list = [
            "{}:{}".format(
                chr_name,
                "".join(["{}:{}".format(x.strip().replace("g.", "").replace(y, ""), y) 
                         for y in re.findall(r"[\D]+", x.strip().replace("g.", ""))]) if "del" not in x else x.strip().replace("g.", "")
                ) 
            for x in nc_code.split(";")[1].strip().replace("[", "").replace("]", ""). replace("'", "").split(",")
        ]
        return ", ".join(position_list)
        
    else:
        return ""
    
df_diplotype_metabolizer = df_diplotype_metabolizer.assign(
    position=[get_position(x) for x in list(df_diplotype_metabolizer["NC_code"].values)])

df_diplotype_guideline = df_diplotype_guideline.assign(
    position=[get_position(x) for x in list(df_diplotype_guideline["NC_code"].values)])

df_haplotype_guideline = df_haplotype_guideline.assign(
    position=[get_position(x) for x in list(df_haplotype_guideline["NC_code"].values)])

df_diplotype_guideline.to_csv("query_file/guideline_diplotype.csv", index=False)
df_haplotype_guideline.to_csv("query_file/guideline_haplotype.csv", index=False)
df_diplotype_metabolizer.to_csv("query_file/guideline_metabolizer.csv", index=False)

In [37]:
group_list = []
for key, content in df_diplotype_metabolizer.groupby(["gene_name", "metabolizer", "organization", "chemical_name"]):
    content_dict = {
        "gene_name": key[0],
        "metabolizer": key[1],
        "organization": key[2],
        "chemical_name": key[3],
        "diplotype": list(set(content["diplotype_name"].values)),
        "haplotype": list(set(content["haplotype_name"].values)),
        "nc_code": list(set(content["NC_code"].values)),
        "ng_code": list(set(content["NG_code"].values)),
        "nucleotide_code": list(set(content["nucleotide_code"].values)),
        "protein_code": list(set(content["protein_code"].values)),
        "recommendation": list(set(content["recommendation"].values))[0],
        "recommendation_chn": list(set(content["recommendation_chn"].values))[0],
        "implication": list(set(content["implication"].values))[0],
        "implication_chn": list(set(content["implication_chn"].values))[0],
        "phenotype": list(set(content["phenotype"].values))[0],
        "link": list(set(content["link"].values))[0],
        "atc_code": list(set(content["atc_code"].values))[0],
        "L1_info": list(set(content["L1_info"].values))[0],
        "L1_info_chn": list(set(content["L1_info_chn"].values))[0],
        "L2_info": list(set(content["L2_info"].values))[0],
        "L2_info_chn": list(set(content["L2_info_chn"].values))[0],
        "L3_info": list(set(content["L3_info"].values))[0],
        "L3_info_chn": list(set(content["L3_info_chn"].values))[0],
        "L4_info": list(set(content["L4_info"].values))[0],
        "L4_info_chn": list(set(content["L4_info_chn"].values))[0],
        "position": list(set([x.strip() for x in ",".join(list(set(content["position"].values))).split(",")])),
        "rsID": list(set([x.strip() for x in ",".join(list(set(content["mapped_rsID"].values))).split(",")]))
    }
    group_list.append(content_dict)
    
pd.DataFrame(group_list).to_csv("query_file/guideline_metabolizer_group.csv", index=False)

In [38]:
group_list = []
for key, content in df_diplotype_guideline.groupby(["gene_name", "organization", "chemical_name", "diplotype_name"]):
    content_dict = {
        "gene_name": key[0],
        "metabolizer": "",
        "organization": key[1],
        "chemical_name": key[2],
        "diplotype": key[3],
        "haplotype": list(set(content["haplotype_name"].values)),
        "nc_code": list(set(content["NC_code"].values)),
        "ng_code": list(set(content["NG_code"].values)),
        "nucleotide_code": list(set(content["nucleotide_code"].values)),
        "protein_code": list(set(content["protein_code"].values)),
        "recommendation": list(set(content["recommendation"].values))[0],
        "recommendation_chn": list(set(content["recommendation_chn"].values))[0],
        "implication": list(set(content["implication"].values))[0],
        "implication_chn": list(set(content["implication_chn"].values))[0],
        "genotype": list(set(content["genotype"].values))[0],
        "genotype_chn": list(set(content["genotype_chn"].values))[0],
        "phenotype": list(set(content["phenotype"].values))[0],
        "link": list(set(content["link"].values))[0],
        "atc_code": list(set(content["atc_code"].values))[0],
        "L1_info": list(set(content["L1_info"].values))[0],
        "L1_info_chn": list(set(content["L1_info_chn"].values))[0],
        "L2_info": list(set(content["L2_info"].values))[0],
        "L2_info_chn": list(set(content["L2_info_chn"].values))[0],
        "L3_info": list(set(content["L3_info"].values))[0],
        "L3_info_chn": list(set(content["L3_info_chn"].values))[0],
        "L4_info": list(set(content["L4_info"].values))[0],
        "L4_info_chn": list(set(content["L4_info_chn"].values))[0],
        "position": list(set([x.strip() for x in ",".join(list(set(content["position"].values))).split(",")])),
        "rsID": list(set([x.strip() for x in ",".join(list(set(content["mapped_rsID"].values))).split(",")]))
    }
    group_list.append(content_dict)
    
pd.DataFrame(group_list).to_csv("query_file/guideline_diplotype_group.csv", index=False)

In [39]:
group_list = []
for key, content in df_haplotype_guideline.groupby(["gene_name", "organization", "chemical_name", "haplotype_name"]):
    content_dict = {
        "gene_name": key[0],
        "metabolizer": "",
        "organization": key[1],
        "chemical_name": key[2],
        "diplotype": "",
        "haplotype": key[3],
        "nc_code": list(set(content["NC_code"].values)),
        "ng_code": list(set(content["NG_code"].values)),
        "nucleotide_code": list(set(content["nucleotide_code"].values)),
        "protein_code": list(set(content["protein_code"].values)),
        "recommendation": list(set(content["recommendation"].values))[0],
        "recommendation_chn": list(set(content["recommendation_chn"].values))[0],
        "implication": list(set(content["implication"].values))[0],
        "implication_chn": list(set(content["implication_chn"].values))[0],
        "genotype": list(set(content["genotype"].values))[0],
        "genotype_chn": list(set(content["genotype_chn"].values))[0],
        "phenotype": list(set(content["phenotype"].values))[0],
        "link": list(set(content["link"].values))[0],
        "atc_code": list(set(content["atc_code"].values))[0],
        "L1_info": list(set(content["L1_info"].values))[0],
        "L1_info_chn": list(set(content["L1_info_chn"].values))[0],
        "L2_info": list(set(content["L2_info"].values))[0],
        "L2_info_chn": list(set(content["L2_info_chn"].values))[0],
        "L3_info": list(set(content["L3_info"].values))[0],
        "L3_info_chn": list(set(content["L3_info_chn"].values))[0],
        "L4_info": list(set(content["L4_info"].values))[0],
        "L4_info_chn": list(set(content["L4_info_chn"].values))[0],
        "position": list(set([x.strip() for x in ",".join(list(set(content["position"].values))).split(",")])),
        "rsID": list(set([x.strip() for x in ",".join(list(set(content["mapped_rsID"].values))).split(",")]))
    }
    group_list.append(content_dict)
    
pd.DataFrame(group_list).to_csv("query_file/guideline_haplotype_group.csv", index=False)

In [41]:
cpic_drug_label_cypher = """
MATCH (ge:gene)-[r:cpic_guideline]->(che:chemical) 
RETURN ge.gene_name as gene_name, ge.OMIM as OMIM, ge.chromosome as chromosome, ge.refseq as refseq,
ge.chromosomal_start_GRCh38 as chromosomal_start, ge.chromosomal_stop_GRCh38 as chromosomal_stop, ge.is_VIP as is_VIP,
r.CPIC_level_status as CPIC_level_status, r.CPIC_level as CPIC_level, r.FDA_PGx_label as FDA_PGx_label,
r.PGKB_evidence_level as PGKB_evidence_level, r.guideline_link as link,
che.chemical_name as chemical_name, che.atc_code as atc_code, 
che.L1_info as L1_info, che.L1_info_chn as L1_info_chn,
che.L2_info as L2_info, che.L2_info_chn as L2_info_chn,
che.L3_info as L3_info, che.L3_info_chn as L3_info_chn,
che.L4_info as L4_info, che.L4_info_chn as L4_info_chn,
"CPIC" as organization
"""

result_list = session.run(cpic_drug_label_cypher).data()
df_cpic_drug_label = pd.DataFrame(result_list).fillna("")

In [42]:
pgkb_drug_label_cypher = """
MATCH (ge:gene)-[r:drug_label]->(che:chemical) 
RETURN ge.gene_name as gene_name, ge.OMIM as OMIM, ge.chromosome as chromosome, ge.refseq as refseq,
ge.chromosomal_start_GRCh38 as chromosomal_start, ge.chromosomal_stop_GRCh38 as chromosomal_stop, ge.is_VIP as is_VIP,
r.link as link, r.label_name as label_name, r.testing_level as testing_level,
che.chemical_name as chemical_name, che.atc_code as atc_code, 
che.L1_info as L1_info, che.L1_info_chn as L1_info_chn,
che.L2_info as L2_info, che.L2_info_chn as L2_info_chn,
che.L3_info as L3_info, che.L3_info_chn as L3_info_chn,
che.L4_info as L4_info, che.L4_info_chn as L4_info_chn,
r.organization as organization
"""

result_list = session.run(pgkb_drug_label_cypher).data()
df_pgkb_drug_label = pd.DataFrame(result_list).fillna("")

In [43]:
df_pgkb_drug_label.to_csv("query_file/pgkb_drug_label.csv", index=False)
df_cpic_drug_label.to_csv("query_file/cpic_drug_label.csv", index=False)

In [44]:
variant_clinical_cypher = """
MATCH (var:variant)-[r:clinical_annotation]->(che:chemical)
WITH var, r, che
MATCH (var)-[:mutation_at]->(ge:gene)
return ge.gene_name as gene_name, ge.chromosome as chromosome,
var.variant_name as variant_name, var.is_reference as is_reference,
var.NC_change_code as NC_code, var.NG_change_code as NG_code, 
var.nucleotide_change_code as nucleotide_code, 
var.protein_change_code as protein_code, var.mapped_rsID as mapped_rsID, 
r.phenotype_category as phenotype_category, r.evidence_level as evidence_level, 
r.level_modifier as level_modifier, r.phenotype as phenotype, 
r.phenotype_chn as phenotype_chn, r.score as score, r.link as link,
che.chemical_name as chemical_name, che.atc_code as atc_code, 
che.L1_info as L1_info, che.L1_info_chn as L1_info_chn,
che.L2_info as L2_info, che.L2_info_chn as L2_info_chn,
che.L3_info as L3_info, che.L3_info_chn as L3_info_chn,
che.L4_info as L4_info, che.L4_info_chn as L4_info_chn
"""

result_list = session.run(variant_clinical_cypher).data()
df_variant_clinical = pd.DataFrame(result_list).fillna("")

In [45]:
group_list = []
for key, content in df_variant_clinical.groupby(["variant_name", "phenotype_category", "chemical_name"]):
    content_dict = {
        "gene_name": list(set(content["gene_name"].values))[0],
        "chromosome": list(set(content["chromosome"].values))[0],
        "variant_name": key[0],
        "phenotype_category": key[1],
        "chemical_name": key[2],
        "nc_code": list(set(content["NC_code"].values)),
        "ng_code": list(set(content["NG_code"].values)),
        "nucleotide_code": list(set(content["nucleotide_code"].values)),
        "protein_code": list(set(content["protein_code"].values)),
        "evidence_level": list(set(content["evidence_level"].values))[0],
        "level_modifier": list(set(content["level_modifier"].values))[0],
        "phenotype": list(set(content["phenotype"].values)),
        "phenotype_chn": list(set(content["phenotype_chn"].values)),
        "link": list(set(content["link"].values))[0],
        "atc_code": list(set(content["atc_code"].values))[0],
        "L1_info": list(set(content["L1_info"].values))[0],
        "L1_info_chn": list(set(content["L1_info_chn"].values))[0],
        "L2_info": list(set(content["L2_info"].values))[0],
        "L2_info_chn": list(set(content["L2_info_chn"].values))[0],
        "L3_info": list(set(content["L3_info"].values))[0],
        "L3_info_chn": list(set(content["L3_info_chn"].values))[0],
        "L4_info": list(set(content["L4_info"].values))[0],
        "L4_info_chn": list(set(content["L4_info_chn"].values))[0],
        "rsID": list(set([x.strip() for x in ",".join(list(set(content["mapped_rsID"].values))).split(",")]))
    }
    group_list.append(content_dict)
df_variant_clinical = pd.DataFrame(group_list)
df_variant_clinical = df_variant_clinical.sort_values(by=["evidence_level"])
df_variant_clinical.index = range(len(df_variant_clinical))
df_variant_clinical.to_csv("query_file/variant_clinical.csv", index=False)

In [5]:
query_str = """
match (hap:haplotype) 
return hap.variant_name as variant_name, hap.NC_change_code as NC_change_code, 
hap.mapped_rsID as mapped_rsID
"""

result_list = session.run(query_str).data()

In [19]:
nc_code_list = list(filter(lambda x: x!= "", list(pd.DataFrame(result_list)["NC_change_code"].values)))

In [28]:
def get_position(nc_code):
    find_list = re.findall(r"chromosome [\dX]+", nc_code)
    if len(find_list) > 0:
        chr_name = find_list[0].replace("chromosome ", "chr")
        position_list = [
            "{}:{}".format(
                chr_name,
                "".join(["{}:{}".format(x.strip().replace("g.", "").replace(y, ""), y) 
                         for y in re.findall(r"[\D]+", x.strip().replace("g.", ""))]) if "del" not in x else x.strip().replace("g.", "")
                ) 
            for x in nc_code.split(";")[1].strip().replace("[", "").replace("]", ""). replace("'", "").split(",")
        ]
        return position_list
        
    else:
        return []

In [32]:
def get_position_new(nc_code):
    chr_name = "".join(re.findall(r"NC_[\d]+.[\d]+", nc_code))
    position_list = ["{}:{}".format(chr_name, x.strip()) 
                     for x in nc_code.split(";")[1].strip().replace("[", "").replace("]", ""). replace("'", "").split(",")]
    return ",".join(position_list)

In [33]:
nc_list = []
for nc in nc_code_list:
    nc_list.extend(get_position_new(nc))

In [34]:
df_mapping = pd.DataFrame(result_list)
df_mapping = df_mapping[(df_mapping["NC_change_code"] != "") & (df_mapping["mapped_rsID"] != "")]

In [37]:
df_mapping = df_mapping.sort_values(by=["variant_name"])
position_list = [get_position_new(x) for x in df_mapping["NC_change_code"].values]
df_mapping = df_mapping.assign(position=position_list)
rsID_list = [",".join(list(filter(lambda x: x!= "", [x.strip() for x in x.split(",")]))) for x in df_mapping["mapped_rsID"].values]
df_mapping = df_mapping.assign(rsID=rsID_list)

In [42]:
df_mapping[["variant_name", "position", "rsID"]].to_csv("processed/position_map.csv", index=False)

In [33]:
from collections import defaultdict

In [34]:
rsID_reverse_dict = defaultdict(list)
rsID_dict = defaultdict(list)
position_erverse_dict = defaultdict(list)
position_dict = defaultdict(list)
for index, row in df_mapping.iterrows():
    for rs in row["rsID"]:
        rsID_reverse_dict[row["variant_name"]].append(rs)
        rsID_dict[rs].append(row["variant_name"])
    for pos in row["position"]:
        position_erverse_dict[row["variant_name"]].append(pos)
        position_dict[pos].append(row["variant_name"])
        


In [36]:
def rsID_to_haplotype(rsID_list):
    result_list = []
    rsID_list = list(filter(lambda x: x in rsID_dict.keys(), rsID_list))
    rs_sub_dict = defaultdict(list)
    
    for rsID in rsID_list:
        rs_sub_dict[rsID].extend(rsID_dict[rsID])
    
    rs_sub_dict = {x: list(set(y)) for x, y in rs_sub_dict.items()}
    candidate_haplotype_list = []
    sub_haplotype_dict = defaultdict(list)
    
    for x, y in rs_sub_dict.items():
        candidate_haplotype_list.extend(y)
        for i in y:
            sub_haplotype_dict[i].append(x)
            
    candidate_haplotype_list = list(set(candidate_haplotype_list))
    
    candidate_dict = {x: rsID_reverse_dict[x] for x in candidate_haplotype_list}
    sub_haplotype_dict = {x: list(set(y)) for x, y in sub_haplotype_dict.items()}
    
    for key in candidate_dict.keys():
        if list(candidate_dict[key]) == list(sub_haplotype_dict[key]):
            result_list.append(key)
            
    return result_list
            
    
def position_to_haplotype(position_list):
    result_list = []
    position_list = list(filter(lambda x: x in position_dict.keys(), position_list))
    pos_sub_list = defaultdict(list)
    
    for pos in position_list:
        pos_sub_list[pos].extend(position_dict[pos])
        
    pos_sub_dict = {x: list(set(y)) for x, y in pos_sub_dict.items()}
    candidate_haplotype_list = []
    sub_haplotype_dict = defaultdict(list)
    
    for x, y in pos_sub_dict.items():
        candidate_haplotype_list.extend(y)
        for i in y:
            sub_haplotype_dict[i].append(x)
            
    candidate_haplotype_list = list(set(candidate_haplotype_list))
    
    candidate_dict = {x: position_erverse_dict[x] for x in candidate_haplotype_list}
    sub_haplotype_dict = {x: list(set(y)) for x, y in sub_haplotype_dict.items()}
    
    for key in candidate_dict.keys():
        if list(candidate_dict[key]) == list(sub_haplotype_dict[key]):
            result_list.append(key)
            
    return result_list
    
    

In [46]:
# generate all drug data
drug_query = """
MATCH (n:drug) RETURN n.drug_name as drug_name
""" 
result_list = session.run(drug_query).data()
df_drug = pd.DataFrame(result_list).fillna("")
df_all_drug = df_drug.drop_duplicates()
df_all_drug = df_all_drug.sort_values(by=["drug_name"])

In [47]:
df_all_drug.to_csv("processed/all_drug.csv", index=False)

In [48]:
# generate pgx drug data
pgx_drug = """
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[]-(v:variant)
return n.drug_name as drug_name, m.chemical_name as chemical_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[]-(dip:diplotype)
return n.drug_name as drug_name, m.chemical_name as chemical_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[]-(ge:gene)
return n.drug_name as drug_name, m.chemical_name as chemical_name
"""
result_list = session.run(pgx_drug).data()
df_pgx_drug = pd.DataFrame(result_list).fillna("")
df_pgx_drug = df_pgx_drug.drop_duplicates()

In [49]:
df_pgx_drug = df_pgx_drug.groupby(["drug_name"])['chemical_name'].apply(lambda x: "; ".join(x)).reset_index(name='chemical_name')

In [52]:
nmpa_drug_query = """
match (n:drug)<-[]-(che:chemical)<-[]-(nm:nmpa_drug)
return distinct che.chemical_name as chemical_name, nm.drug_name_chn as drug_name_chn, 
nm.drug_name_eng as drug_name_eng, 
nm.drug_code as drug_code, nm.license as license, 
nm.chn_business_name as chn_business_name, nm.eng_business_name as eng_business_name,
nm.country as country, nm.company as company 
order by drug_name_chn
"""

result_list = session.run(nmpa_drug_query).data()
df_nmpa_drug = pd.DataFrame(result_list).fillna("")
df_nmpa_count = df_nmpa_drug.groupby(["drug_name_chn"])["drug_code"].count().reset_index(name='nmpa_drug_count')

In [53]:
df_merge = pd.merge(df_pgx_drug, df_nmpa_count, how="left", left_on=["drug_name"], right_on=["drug_name_chn"])
df_merge["nmpa_drug_count"] = df_merge["nmpa_drug_count"].fillna(0).astype(int)
df_merge = df_merge[["drug_name", "chemical_name", "nmpa_drug_count"]]
df_merge = df_merge.sort_values(by=["drug_name"])

In [54]:
df_merge.to_csv("processed/pgx_drug_stats.csv", index=False)

In [55]:
# generate clinical data
pgx_clinical_drug = """
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:clinical_annotation]-(v:variant)
return n.drug_name as drug_name, m.chemical_name as chemical_name, v.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:clinical_annotation]-(dip:diplotype)
return n.drug_name as drug_name, m.chemical_name as chemical_name, dip.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:clinical_annotation]-(ge:gene)
return n.drug_name as drug_name, m.chemical_name as chemical_name, ge.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:cpic_guideline]-(v:variant)
return n.drug_name as drug_name, m.chemical_name as chemical_name, v.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:cpic_guideline]-(dip:diplotype)
return n.drug_name as drug_name, m.chemical_name as chemical_name, dip.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:cpic_guideline]-(ge:gene)
return n.drug_name as drug_name, m.chemical_name as chemical_name, ge.display as v_name
"""
result_list = session.run(pgx_clinical_drug).data()
pgx_clinical_drug = pd.DataFrame(result_list).fillna("")
pgx_clinical_drug = pgx_clinical_drug.drop_duplicates()

In [56]:
pgx_clinical_drug = pgx_clinical_drug.groupby(["drug_name"])["v_name"].count().reset_index(name='clinical_relation_count')
pgx_clinical_drug = pgx_clinical_drug.sort_values(by=["drug_name"])

In [57]:
pgx_clinical_drug.to_csv("processed/pgx_drug_clinical_stats.csv", index=False)

In [58]:
# generate clinical 1A 1B data
pgx_clinical_drug_1A_1B = """
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:clinical_annotation]-(v:variant)
where r.evidence_level =~ "1[A|B]"
return n.drug_name as drug_name, m.chemical_name as chemical_name, v.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:clinical_annotation]-(dip:diplotype)
where r.evidence_level =~ "1[A|B]"
return n.drug_name as drug_name, m.chemical_name as chemical_name, dip.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:clinical_annotation]-(ge:gene)
where r.evidence_level =~ "1[A|B]"
return n.drug_name as drug_name, m.chemical_name as chemical_name, ge.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:cpic_guideline]-(v:variant)
where r.CPIC_level = "A" or r.PGKB_evidence_level =~ "1[A|B]"
return n.drug_name as drug_name, m.chemical_name as chemical_name, v.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:cpic_guideline]-(dip:diplotype)
where r.CPIC_level = "A" or r.PGKB_evidence_level =~ "1[A|B]"
return n.drug_name as drug_name, m.chemical_name as chemical_name, dip.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:cpic_guideline]-(ge:gene)
where r.CPIC_level = "A" or r.PGKB_evidence_level =~ "1[A|B]"
return n.drug_name as drug_name, m.chemical_name as chemical_name, ge.display as v_name
"""
result_list = session.run(pgx_clinical_drug_1A_1B).data()
pgx_clinical_drug_1A_1B = pd.DataFrame(result_list).fillna("")
pgx_clinical_drug_1A_1B = pgx_clinical_drug_1A_1B.drop_duplicates()

In [59]:
pgx_clinical_drug_1A_1B = pgx_clinical_drug_1A_1B.groupby(["drug_name"])["v_name"].count().reset_index(name='clinical_relation_count')
pgx_clinical_drug_1A_1B = pgx_clinical_drug_1A_1B.sort_values(by=["drug_name"])

In [60]:
pgx_clinical_drug_1A_1B.to_csv("processed/pgx_drug_clinical_stats_1A_1B.csv", index=False)

In [75]:
# generate metabolizer data
pgx_metabolizer_query = """
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:diplotype_metabolizer]-(v:variant)
return n.drug_name as drug_name, m.chemical_name as chemical_name, v.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:diplotype_metabolizer]-(dip:diplotype)
return n.drug_name as drug_name, m.chemical_name as chemical_name, dip.display as v_name
"""

result_list = session.run(pgx_metabolizer_query).data()
pgx_meta_drug = pd.DataFrame(result_list).fillna("")
pgx_meta_drug = pgx_meta_drug.drop_duplicates()

In [78]:
df_pgx_meta_drug = pgx_meta_drug.groupby(["drug_name"])["drug_name"].count().reset_index(name='metabolizer_relation_count')
df_pgx_meta_drug = df_pgx_meta_drug.sort_values(by=["drug_name"])

In [80]:
df_pgx_meta_drug.to_csv("processed/pgx_drug_metabolizer_stats.csv", index=False)

In [61]:
# generate pgx research label data
pgx_research_drug = """
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:research_annotation]-(v:variant)
return n.drug_name as drug_name, m.chemical_name as chemical_name, v.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:research_annotation]-(dip:diplotype)
return n.drug_name as drug_name, m.chemical_name as chemical_name, dip.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:research_annotation]-(ge:gene)
return n.drug_name as drug_name, m.chemical_name as chemical_name, ge.display as v_name
"""

result_list = session.run(pgx_research_drug).data()
pgx_research_drug = pd.DataFrame(result_list).fillna("")
pgx_research_drug = pgx_research_drug.drop_duplicates()

In [62]:
df_pgx_research_drug = pgx_research_drug.groupby(["drug_name"])["drug_name"].count().reset_index(name='research_relation_count')
df_pgx_research_drug = df_pgx_research_drug.sort_values(by=["drug_name"])

In [63]:
df_pgx_research_drug.to_csv("processed/pgx_drug_research_stats.csv", index=False)

In [72]:
# generate pgx drug_label data
# "Swissmedic" "EMA" "FDA" "PMDA" "HCSC"
pgx_drug_label_drug = """
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:drug_label]-(v:variant)
return n.drug_name as drug_name, m.chemical_name as chemical_name, v.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:drug_label]-(dip:diplotype)
return n.drug_name as drug_name, m.chemical_name as chemical_name, dip.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:drug_label]-(ge:gene)
return n.drug_name as drug_name, m.chemical_name as chemical_name, ge.display as v_name
"""
result_list = session.run(pgx_drug_label_drug).data()
pgx_drug_label_drug = pd.DataFrame(result_list).fillna("")
pgx_drug_label_drug = pgx_drug_label_drug.drop_duplicates()

In [73]:
pgx_drug_label_drug = pgx_drug_label_drug.groupby(["drug_name"])["drug_name"].count().reset_index(name='label_relation_count')
pgx_drug_label_drug = pgx_drug_label_drug.sort_values(by=["drug_name"])

In [74]:
pgx_drug_label_drug.to_csv("processed/pgx_drug_label_stats.csv", index=False)

In [75]:
# generate pgx drug_label data
# "FDA"
pgx_FDA_drug_label_drug = """
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:drug_label]-(v:variant)
where r.organization = "FDA"
return n.drug_name as drug_name, m.chemical_name as chemical_name, v.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:drug_label]-(dip:diplotype)
where r.organization = "FDA"
return n.drug_name as drug_name, m.chemical_name as chemical_name, dip.display as v_name
union
match (n:drug)<-[:chemical_drug_relation]-(m:chemical)<-[r:drug_label]-(ge:gene)
where r.organization = "FDA"
return n.drug_name as drug_name, m.chemical_name as chemical_name, ge.display as v_name
"""
result_list = session.run(pgx_FDA_drug_label_drug).data()
pgx_FDA_drug_label_drug = pd.DataFrame(result_list).fillna("")
pgx_FDA_drug_label_drug = pgx_FDA_drug_label_drug.drop_duplicates()

In [76]:
pgx_FDA_drug_label_drug = pgx_FDA_drug_label_drug.groupby(["drug_name"])["drug_name"].count().reset_index(name='label_relation_count')
pgx_FDA_drug_label_drug = pgx_FDA_drug_label_drug.sort_values(by=["drug_name"])

In [77]:
pgx_FDA_drug_label_drug.to_csv("processed/pgx_fda_drug_label_stats.csv", index=False)

In [4]:
# generate neo4j all relation pairs to train embedding.
tuplet_query = """
match (n)-[r]->(m)
return id(n) as start_node_id, type(r) as relation, id(m) as end_node_id
"""
result_list = session.run(tuplet_query).data()


In [5]:
df_tuplet = pd.DataFrame(result_list)
df_tuplet.to_csv("train/triple.csv", index=False)

In [6]:
relation_list = list(sorted(set(df_tuplet["relation"].values)))
pd.DataFrame({"id": range(len(relation_list)), "relation": relation_list}).to_csv("train/relation.csv", index=False)

In [7]:
# generate node name2id
node_query = """
match (n)
return id(n) as node_id, n.display as name, labels(n) as labels
"""
result_list = session.run(node_query).data()

In [8]:
df_node = pd.DataFrame(result_list)
labels = [x[1] if "variant" in x else x[0] for x in df_node["labels"].values]
df_node = df_node.assign(labels=labels)
df_node.to_csv("train/node.csv", index=False)

In [79]:
import json
with open("../disease-kb/processed/drug_dict.json", "r") as f:
    drug_dict = json.load(f)

In [80]:
chinese_drug_list = list(drug_dict.keys())[:6555]
chinese_drug_list.extend(["二十九味能消散", "肺宁片", "百合固金片", "维C银翘胶囊", "红草止鼾胶囊", "红草止鼾颗粒"
                          "利肺胶囊", "复方鱼腥草片", "复方鱼腥草滴丸", "参龙宁心胶囊", "仁青芒觉", "足爽", "开塞露"
                          "济泰片", "福康片"])

In [81]:
drug_list = [[x, "中成药" if x in chinese_drug_list or 
  any(True if y in x else False for y in ["红景天", "丹参", "清凉散", "鼻炎宁", "复方", "散"
                                          "红草", "云芝", "参皇", "天草", "龙骨", "维血康",
                                             "心通"]) else "西药"] 
 for x in list(df_all_drug["drug_name"].values)]

In [82]:
pd.DataFrame(drug_list, columns=["drug_name", "西药/中成药"]).to_csv("processed/all_drug_class.csv", index=False)